# Hansa
## Configuration and Hyperparameters

In [ ]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
import time
from datetime import datetime

SEED = 69
np.random.seed(SEED)
tf.random.set_seed(SEED)

DATA_DIR = os.path.join('..', 'data', 'train')
TEST_DIR = os.path.join('..', 'data', 'test')
IMG_SIZE = (32, 32)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

LEARNING_RATE = 0.00001
EPOCHS = 20
DROPOUT_RATE = 0.5
L2_REGULARIZATION = 0.001

EARLY_STOPPING_PATIENCE = 3
REDUCE_LR_PATIENCE = 2
REDUCE_LR_FACTOR = 0.5

MODEL_NAME = 'hansa_best.keras'
HISTORY_PLOT = 'training_history.png'
CONFUSION_MATRIX_PLOT = 'confusion_matrix.png'

print(f"Configuration loaded at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Data Loading and Preprocessing

In [ ]:
print("Loading Training Data...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

print("\nLoading Validation Data...")
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

print("\nLoading Test Data...")
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=False 
)

class_names = train_ds.class_names
print(f"\nClasses: {class_names}")

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("\nData pipeline optimized for efficient training.")

## Model Architecture
### CNN with Batch Normalization, Dropout, and L2 Regularization

In [ ]:
from tensorflow.keras import layers, models, regularizers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    
    data_augmentation,
    
    layers.Rescaling(1./255),

    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    layers.GlobalAveragePooling2D(),

    layers.Dense(128, activation='relu', 
                 kernel_regularizer=regularizers.l2(L2_REGULARIZATION)),
    layers.Dropout(DROPOUT_RATE),
    layers.Dense(1, activation='sigmoid')
])

print("\n" + "="*50)
print("MODEL ARCHITECTURE")
print("="*50)
model.summary()
print("="*50)

## Model Compilation

In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', 
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

print("Model compiled successfully!")

## Training Callbacks Setup

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=EARLY_STOPPING_PATIENCE,
    restore_best_weights=True,
    verbose=2
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    MODEL_NAME,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=2
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=REDUCE_LR_FACTOR,
    patience=REDUCE_LR_PATIENCE,
    min_lr=1e-7,
    verbose=2
)

callbacks = [early_stopping, checkpoint, reduce_lr]
print("Callbacks configured:")
print("- Early Stopping")
print("- Model Checkpoint")
print("- Learning Rate Reduction")

## Model Training

In [ ]:
print("\n" + "="*50)
print("STARTING TRAINING")
print("="*50)
print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Learning Rate: {LEARNING_RATE}")
print("="*50 + "\n")

start_time = time.time()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=2
)

training_time = time.time() - start_time

print("\n" + "="*50)
print("TRAINING COMPLETED")
print("="*50)
print(f"Total Training Time: {training_time/60:.2f} minutes")
print(f"Average Time per Epoch: {training_time/len(history.history['loss']):.2f} seconds")
print("="*50)

## Training History Visualization

In [ ]:
def plot_training_history(history):
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs_range = range(len(history.history['accuracy']))
    
    axes[0, 0].plot(epochs_range, history.history['accuracy'], label='Training')
    axes[0, 0].plot(epochs_range, history.history['val_accuracy'], label='Validation')
    axes[0, 0].set_title('Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].plot(epochs_range, history.history['loss'], label='Training')
    axes[0, 1].plot(epochs_range, history.history['val_loss'], label='Validation')
    axes[0, 1].set_title('Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].plot(epochs_range, history.history['precision'], label='Training')
    axes[1, 0].plot(epochs_range, history.history['val_precision'], label='Validation')
    axes[1, 0].set_title('Precision', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].plot(epochs_range, history.history['recall'], label='Training')
    axes[1, 1].plot(epochs_range, history.history['val_recall'], label='Validation')
    axes[1, 1].set_title('Recall', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(HISTORY_PLOT, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nTraining history plot saved as '{HISTORY_PLOT}'")

plot_training_history(history)

## Model Evaluation on Test Set

In [ ]:
print("\n" + "="*50)
print("EVALUATING ON TEST DATA")
print("="*50)

test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds, verbose=1)
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print("\n" + "="*50)
print("TEST SET RESULTS")
print("="*50)
print(f"Loss:      {test_loss:.4f}")
print(f"Accuracy:  {test_acc*100:.2f}%")
print(f"Precision: {test_precision*100:.2f}%")
print(f"Recall:    {test_recall*100:.2f}%")
print(f"F1-Score:  {test_f1*100:.2f}%")
print("="*50)

## Confusion Matrix and ROC Curve

In [ ]:
y_pred_probs = model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

y_true = np.concatenate([y for x, y in test_ds], axis=0)

cm = confusion_matrix(y_true, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=class_names, yticklabels=class_names)
axes[0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs)
roc_auc = auc(fpr, tpr)

axes[1].plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.2f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[1].legend(loc="lower right")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(CONFUSION_MATRIX_PLOT, dpi=300, bbox_inches='tight')
plt.show()

print(f"\nConfusion matrix and ROC curve saved as '{CONFUSION_MATRIX_PLOT}'")
print(f"AUC Score: {roc_auc:.4f}")

## Detailed Classification Report

In [ ]:
print("\n" + "="*50)
print("CLASSIFICATION REPORT")
print("="*50)
print(classification_report(y_true, y_pred, target_names=class_names))
print("="*50)

## Sample Predictions Visualization

In [ ]:
def show_sample_predictions(dataset, model, num_samples=9):

    plt.figure(figsize=(15, 15))
    
    images, labels = next(iter(dataset))
    predictions = model.predict(images[:num_samples])
    
    for i in range(num_samples):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        pred_class = 1 if predictions[i] > 0.5 else 0
        true_class = int(labels[i])
        confidence = predictions[i][0] if pred_class == 1 else 1 - predictions[i][0]
        
        color = 'green' if pred_class == true_class else 'red'
        
        plt.title(f"True: {class_names[true_class]}\n"
                 f"Pred: {class_names[pred_class]} ({confidence*100:.1f}%)",
                 color=color, fontweight='bold')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Sample predictions saved as 'sample_predictions.png'")

show_sample_predictions(test_ds, model)

## Save Final Model and Configuration

In [ ]:
import json

model.save('hansa.keras')
print(f"Final model saved as 'hansa.keras'")

config = {
    'img_size': IMG_SIZE,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'epochs_trained': len(history.history['loss']),
    'final_train_accuracy': float(history.history['accuracy'][-1]),
    'final_val_accuracy': float(history.history['val_accuracy'][-1]),
    'test_accuracy': float(test_acc),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_f1': float(test_f1),
    'auc_score': float(roc_auc),
    'class_names': class_names,
    'training_time_minutes': training_time/60,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('training_config.json', 'w') as f:
    json.dump(config, f, indent=4)

print("Training configuration saved as 'training_config.json'")
print("\n" + "="*50)
print("ALL DONE! 🎉")
print("="*50)